In [1]:
from src.data.dataloaders import GMC_DataLoader, NIH_DataLoader
from src.models.train import train
from src.models.evaluate import test
from src.models.CheXNet import DenseNet121
from src.models.baseline import Baseline
from src.models.unet import UNet
from src.models.style_encoder import ConvEncoder
from src.models.super_model import SuperModel
from src.models.resnet import get_resnet

from torch import optim
import torch

%load_ext autoreload
%autoreload 2

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
train_loader_GMC = GMC_DataLoader(stage='train', batch_size = 4, shuffle=True, num_workers=0, pin_memory=True,)
train_loader_NIH= NIH_DataLoader(stage='train', batch_size = 4, shuffle=True, num_workers=0, pin_memory=True,)
train_loaders = [train_loader_GMC, train_loader_NIH]

test_loader_GMC = GMC_DataLoader(stage='test', batch_size = 4, shuffle=True, num_workers=0, pin_memory=True,)
test_loader_NIH= NIH_DataLoader(stage='test', batch_size = 4, shuffle=True, num_workers=0, pin_memory=True,)
test_loaders = [test_loader_GMC, test_loader_NIH]


In [3]:
super_model = SuperModel()
optimizer = optim.Adam(super_model.parameters(), lr=0.002)
criterion = torch.nn.BCELoss()

In [4]:
n_epochs = 20
irm = 0
vrex = 0

train(super_model, train_loaders, test_loaders, optimizer, criterion, irm, vrex, n_epochs, device, exp2 = True)

  0%|          | 10/26080 [01:20<62:18:10,  8.60s/it]

In [ ]:
for i, (batch_x, batch_y) in enumerate(train_loader_GMC):
    print(batch_x.shape, batch_y.shape)

    pred = super_model(batch_x)

    if i == 5:
        break

torch.Size([4, 3, 224, 224]) torch.Size([4, 1])
torch.Size([4, 3, 224, 224]) torch.Size([4, 1])
torch.Size([4, 3, 224, 224]) torch.Size([4, 1])
torch.Size([4, 3, 224, 224]) torch.Size([4, 1])
torch.Size([4, 3, 224, 224]) torch.Size([4, 1])
torch.Size([4, 3, 224, 224]) torch.Size([4, 1])
